### Product Analysis
---
    1. What are the top 3 products by total revenue before discount?

In [1]:
SELECT TOP 3
    prod_id,
    product_name,
    SUM(qty*s.price) as revenue
FROM sales s 
JOIN product_details p ON p.product_id = s.prod_id
GROUP BY prod_id, product_name
ORDER BY revenue DESC

(3 rows affected)

Total execution time: 00:00:00.012

prod_id,product_name,revenue
2a2353,Blue Polo Shirt - Mens,217683
9ec847,Grey Fashion Jacket - Womens,209304
5d267b,White Tee Shirt - Mens,152000


    2. What is the total quantity, revenue and discount for each segment?

In [2]:
SELECT
    segment_name,
    SUM(qty) AS quantity,
    SUM(qty*s.price) AS revenue,
    SUM(qty*s.price*discount)/100 AS discount
FROM sales s 
JOIN product_details p ON p.product_id = s.prod_id
GROUP BY segment_name

(4 rows affected)

Total execution time: 00:00:00.019

segment_name,quantity,revenue,discount
Jacket,11385,366983,44277
Jeans,11349,208350,25343
Shirt,11265,406143,49594
Socks,11217,307977,37013


    3. What is the top selling product for each segment?

In [3]:
WITH tbl AS(
    SELECT
        segment_name,
        product_name,
        SUM(qty) AS quantity,
        SUM(qty*s.price) AS revenue,
        RANK() OVER(PARTITION BY segment_name ORDER BY SUM(qty) DESC) AS rank_qty
    FROM sales s 
    JOIN product_details p ON p.product_id = s.prod_id
    GROUP BY segment_name, product_name
)
SELECT 
    segment_name,
    product_name,
    quantity,
    revenue
FROM tbl
WHERE rank_qty = 1

(4 rows affected)

Total execution time: 00:00:00.016

segment_name,product_name,quantity,revenue
Jacket,Grey Fashion Jacket - Womens,3876,209304
Jeans,Navy Oversized Jeans - Womens,3856,50128
Shirt,Blue Polo Shirt - Mens,3819,217683
Socks,Navy Solid Socks - Mens,3792,136512


    4. What is the total quantity, revenue and discount for each category?

In [4]:
SELECT
    category_name,
    SUM(qty) AS quantity,
    SUM(qty*s.price) AS revenue,
    SUM(qty*s.price*discount)/100 AS discount
FROM sales s 
JOIN product_details p ON p.product_id = s.prod_id
GROUP BY category_name

(2 rows affected)

Total execution time: 00:00:00.017

category_name,quantity,revenue,discount
Mens,22482,714120,86607
Womens,22734,575333,69621


    5. What is the top selling product for each category?

In [5]:
WITH tbl AS(
    SELECT
        category_name,
        product_name,
        SUM(qty) AS quantity,
        SUM(qty*s.price) AS revenue,
        RANK() OVER(PARTITION BY category_name ORDER BY SUM(qty) DESC) AS rank_qty
    FROM sales s 
    JOIN product_details p ON p.product_id = s.prod_id
    GROUP BY category_name, product_name
)
SELECT
    category_name,
    product_name,
    quantity,
    revenue
FROM tbl
WHERE rank_qty = 1

(2 rows affected)

Total execution time: 00:00:00.019

category_name,product_name,quantity,revenue
Mens,Blue Polo Shirt - Mens,3819,217683
Womens,Grey Fashion Jacket - Womens,3876,209304


    6. What is the percentage split of revenue by product for each segment?

In [6]:
WITH tbl AS(
    SELECT DISTINCT 
        segment_name,
        product_name,
        SUM(qty*s.price) OVER(PARTITION BY segment_name, product_name) AS sum_per_product,
        SUM(qty*s.price) OVER(PARTITION BY segment_name) AS sum_per_segment
    FROM sales s 
    JOIN product_details p ON p.product_id = s.prod_id
)
SELECT *,
    ROUND(100*CAST(sum_per_product as float)/sum_per_segment,2) AS percent_revenue_by_product_each_segment
FROM tbl
ORDER BY segment_name

(12 rows affected)

Total execution time: 00:00:00.143

segment_name,product_name,sum_per_product,sum_per_segment,percent_revenue_by_product_each_segment
Jacket,Grey Fashion Jacket - Womens,209304,366983,57.03
Jacket,Indigo Rain Jacket - Womens,71383,366983,19.45
Jacket,Khaki Suit Jacket - Womens,86296,366983,23.51
Jeans,Black Straight Jeans - Womens,121152,208350,58.15
Jeans,Cream Relaxed Jeans - Womens,37070,208350,17.79
Jeans,Navy Oversized Jeans - Womens,50128,208350,24.06
Shirt,Blue Polo Shirt - Mens,217683,406143,53.6
Shirt,Teal Button Up Shirt - Mens,36460,406143,8.98
Shirt,White Tee Shirt - Mens,152000,406143,37.43
Socks,Navy Solid Socks - Mens,136512,307977,44.33


    7. What is the percentage split of revenue by segment for each category?

In [7]:
WITH tbl AS(
    SELECT DISTINCT 
        category_name,
        segment_name,
        SUM(qty*s.price) OVER(PARTITION BY category_name, segment_name) AS sum_per_segment,
        SUM(qty*s.price) OVER(PARTITION BY category_name) AS sum_per_category
    FROM sales s 
    JOIN product_details p ON p.product_id = s.prod_id
)
SELECT *,
    ROUND(100*CAST(sum_per_segment as float)/sum_per_category,2) AS percent_revenue_by_segment_each_category
FROM tbl
ORDER BY category_name

(4 rows affected)

Total execution time: 00:00:00.146

category_name,segment_name,sum_per_segment,sum_per_category,percent_revenue_by_segment_each_category
Mens,Shirt,406143,714120,56.87
Mens,Socks,307977,714120,43.13
Womens,Jacket,366983,575333,63.79
Womens,Jeans,208350,575333,36.21


    8. What is the percentage split of total revenue by category?

In [8]:
SELECT DISTINCT 
    category_name,
    SUM(qty*s.price) OVER(PARTITION BY category_name) AS sum_per_category,
    ROUND(100*CAST(SUM(qty*s.price) OVER(PARTITION BY category_name) as float)/(SELECT SUM(qty*price) FROM sales),2) 
    AS percentage_revenue_by_category
FROM sales s 
JOIN product_details p ON p.product_id = s.prod_id


(2 rows affected)

Total execution time: 00:00:00.107

category_name,sum_per_category,percentage_revenue_by_category
Womens,575333,44.62
Mens,714120,55.38


    9. What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)

In [9]:
SELECT
    product_name,
    100*CAST(COUNT(*) AS float)/(SELECT COUNT(DISTINCT txn_id) FROM sales) AS transaction_penetration
FROM sales s 
JOIN product_details p ON p.product_id = s.prod_id
GROUP BY product_name

(12 rows affected)

Total execution time: 00:00:00.031

product_name,transaction_penetration
Black Straight Jeans - Womens,49.84
Blue Polo Shirt - Mens,50.72
Cream Relaxed Jeans - Womens,49.72
Grey Fashion Jacket - Womens,51
Indigo Rain Jacket - Womens,50
Khaki Suit Jacket - Womens,49.88
Navy Oversized Jeans - Womens,50.96
Navy Solid Socks - Mens,51.24
Pink Fluro Polkadot Socks - Mens,50.32
Teal Button Up Shirt - Mens,49.68


    10. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

In [10]:
WITH tbl AS (   
    SELECT  
        txn_id, 
        p1.product_name
    FROM sales s 
    LEFT JOIN product_details  p1 ON s.prod_id = p1.product_id
)
SELECT TOP 1
    t1.product_name AS PRODUCT_1,
    t2.product_name AS PRODUCT_2,
    t3.product_name AS PRODUCT_3,
    COUNT (*) AS time_trans
FROM tbl t1 
JOIN tbl t2 ON t1.txn_id = t2.txn_id AND t1.product_name < t2.product_name
JOIN tbl t3 ON t1.txn_id = t3.txn_id AND t1.product_name < t3.product_name AND t2.product_name < t3.product_name
GROUP BY t1.product_name, t2.product_name, t3.product_name
ORDER BY time_trans DESC ;

(1 row affected)

Total execution time: 00:00:00.325

PRODUCT_1,PRODUCT_2,PRODUCT_3,time_trans
Grey Fashion Jacket - Womens,Teal Button Up Shirt - Mens,White Tee Shirt - Mens,352
